# Exploration, Cleaning

In [1]:
import pandas as pd
import numpy as np


In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/buildweek-kickstarter/data_science/master/data/'

In [3]:
import pandas as pd

df = pd.read_csv('kickstarter - Sheet1.csv')

print(df.shape)
df

(999, 13)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 0:20:50,45000,2013-01-12 0:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 4:24:11,5000,2012-03-17 3:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 1:00:00,19500,2015-07-04 8:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1005848496,The 1st Motion Tracking DIY Smart Home Securit...,Gadgets,Technology,USD,2016-12-31 3:54:32,25000,2016-11-01 2:54:32,58978,live,397,US,7235
995,1005862247,Veterans,Photography,Photography,USD,2012-08-15 6:00:00,5000,2012-07-09 5:39:06,5637,successful,87,US,5637
996,1005863674,MY VERY FIRST KICKSTARTER,Movie Theaters,Film & Video,USD,2014-08-12 20:08:35,6500,2014-07-13 20:08:35,27,failed,2,US,27
997,1005870420,This Song Is About You,Music,Music,USD,2014-07-11 23:35:00,40000,2014-06-11 23:35:00,0,failed,0,US,0


In [4]:
def normalise_row(row):
    if row['state'] == 'successful':
      return 'successful'
    if row['state'] == 'live':
      return 'live'
    else:
      return 'failed'

In [5]:
df['state'] = df.apply(lambda row : normalise_row(row), axis=1)

In [7]:
df['state'].value_counts()

failed        604
successful    377
live           18
Name: state, dtype: int64

In [8]:
df['launched'][0]

'2015-08-11 12:12:28'

In [6]:
liverows = df[df['state'] == 'live'].index
 
# Delete these row indexes from dataFrame
df.drop(liverows , inplace=True)

In [10]:
df

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 0:20:50,45000,2013-01-12 0:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 4:24:11,5000,2012-03-17 3:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 1:00:00,19500,2015-07-04 8:35:03,1283,failed,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,1005843591,"""Projection"" a short film",Shorts,Film & Video,USD,2011-06-18 2:43:00,5000,2011-05-03 22:19:58,1367,failed,20,US,1367
995,1005862247,Veterans,Photography,Photography,USD,2012-08-15 6:00:00,5000,2012-07-09 5:39:06,5637,successful,87,US,5637
996,1005863674,MY VERY FIRST KICKSTARTER,Movie Theaters,Film & Video,USD,2014-08-12 20:08:35,6500,2014-07-13 20:08:35,27,failed,2,US,27
997,1005870420,This Song Is About You,Music,Music,USD,2014-07-11 23:35:00,40000,2014-06-11 23:35:00,0,failed,0,US,0


In [11]:
df['deadline'].dtypes


dtype('O')

In [7]:
# Split train into train & val
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, train_size=0.80, test_size=0.20, 
                              stratify=df['state'], random_state=42)

In [8]:
train

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
886,1005205097,We Came To Sweat,Dance,Dance,USD,2011-05-03 5:59:00,2223,2011-04-11 9:17:10,2845,successful,97,US,2845
301,1001560762,""" The Patch, Organics """,Comics,Comics,USD,2014-12-23 3:21:18,65000,2014-11-23 3:21:18,50,failed,2,US,50
728,1004112120,The MagPi Magazine - From Virtual to Reality,Journalism,Journalism,GBP,2012-12-31 20:01:04,6000,2012-11-29 20:01:04,"29,576.33",successful,616,GB,"47,368.74"
72,1000331311,AWE - Antediluvian Wars: Extermination Tactica...,Tabletop Games,Games,USD,2014-08-31 0:00:00,13000,2014-08-01 0:02:17,1811,failed,34,US,1811
636,1003539829,Unconditional Hope - A Survivors Story.,Documentary,Film & Video,USD,2012-12-29 2:00:00,40000,2012-11-26 21:14:51,45155,successful,411,US,45155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1001064677,Equinox: my first comic (genre: fantasy),Comics,Comics,USD,2011-06-04 21:36:40,5150,2011-04-25 21:36:40,"5,965.50",successful,97,US,"5,965.50"
235,1001181420,Sleeping in a Truck,Rock,Music,USD,2009-07-15 4:58:00,10000,2009-06-20 22:15:06,10328,successful,65,US,10328
476,100264379,P-LINK : Guitar Pickup Interchange System,Product Design,Design,AUD,2016-09-30 4:00:00,75000,2016-08-18 16:31:22,13863,failed,89,AU,"1,139.94"
192,1000937140,The FUN n' EASY CookBook For Children's Well.....,Publishing,Publishing,USD,2012-12-09 17:59:00,3500,2012-11-07 19:53:01,0,failed,0,US,0


In [9]:
val

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
737,1004192678,VINCE DURDEN: World's Most Eco-Friendly Backpack,Accessories,Fashion,USD,2016-11-11 20:17:59,2000,2016-10-12 20:17:59,5189,successful,77,US,886
397,1002182970,iPhone Skin Made From Real Apple Wood,Product Design,Design,USD,2012-06-28 16:48:45,500,2012-05-29 16:48:45,838.01,successful,31,US,838.01
33,1000132945,Zen in America: a film on Zen Buddhism in Nort...,Documentary,Film & Video,USD,2013-10-21 23:00:54,10000,2013-09-21 23:00:54,15663,successful,213,US,15663
570,1003202814,TALUVA: DELUXE VERSION,Tabletop Games,Games,USD,2015-01-02 9:17:22,15000,2014-12-03 9:17:22,"103,114.50",successful,637,US,"103,114.50"
292,1001516983,Baby Boom Baby Bag,Fashion,Fashion,GBP,2013-09-28 13:33:00,1500,2013-08-28 18:58:44,279,failed,12,GB,434.3548195
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
34,1000134913,Galaxy: Winnie and Friends,Illustration,Art,USD,2016-01-27 21:47:29,3000,2015-12-23 21:47:29,17,failed,2,US,17
17,1000071625,Boco Tea,Food,Food,USD,2012-06-02 17:24:32,5000,2012-05-03 17:24:32,1781,failed,40,US,1781
969,1005696636,GBS Detroit Presents Daniel Harrison and the $...,Indie Rock,Music,USD,2013-06-08 5:59:00,1200,2013-05-23 21:05:24,1525,successful,22,US,1525


In [11]:
# added new features to our data and got rid of duplicates
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
 
    # Stop CopyWarning
    X = X.copy()
    
   
    

    # converting date recorderd into int and making new feature
    X['deadline'] = pd.to_datetime(X['deadline'])
    # X['deadline'] = X['deadline'].dt
    # X['deadline'] =X['deadline'].astype(int)

    X['launched'] = pd.to_datetime(X['launched'])
    # X['launched'] = X['launched'].dt
    # X['launched'] =X['launched'].astype(int)

    X['length_of_campaign'] = (X['deadline'] - X['launched']).dt.days   

    return X
    


In [12]:
train = wrangle(train)
val = wrangle(val)

In [13]:
train

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,length_of_campaign
886,1005205097,We Came To Sweat,Dance,Dance,USD,2011-05-03 05:59:00,2223,2011-04-11 09:17:10,2845,successful,97,US,2845,21
301,1001560762,""" The Patch, Organics """,Comics,Comics,USD,2014-12-23 03:21:18,65000,2014-11-23 03:21:18,50,failed,2,US,50,30
728,1004112120,The MagPi Magazine - From Virtual to Reality,Journalism,Journalism,GBP,2012-12-31 20:01:04,6000,2012-11-29 20:01:04,"29,576.33",successful,616,GB,"47,368.74",32
72,1000331311,AWE - Antediluvian Wars: Extermination Tactica...,Tabletop Games,Games,USD,2014-08-31 00:00:00,13000,2014-08-01 00:02:17,1811,failed,34,US,1811,29
636,1003539829,Unconditional Hope - A Survivors Story.,Documentary,Film & Video,USD,2012-12-29 02:00:00,40000,2012-11-26 21:14:51,45155,successful,411,US,45155,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,1001064677,Equinox: my first comic (genre: fantasy),Comics,Comics,USD,2011-06-04 21:36:40,5150,2011-04-25 21:36:40,"5,965.50",successful,97,US,"5,965.50",40
235,1001181420,Sleeping in a Truck,Rock,Music,USD,2009-07-15 04:58:00,10000,2009-06-20 22:15:06,10328,successful,65,US,10328,24
476,100264379,P-LINK : Guitar Pickup Interchange System,Product Design,Design,AUD,2016-09-30 04:00:00,75000,2016-08-18 16:31:22,13863,failed,89,AU,"1,139.94",42
192,1000937140,The FUN n' EASY CookBook For Children's Well.....,Publishing,Publishing,USD,2012-12-09 17:59:00,3500,2012-11-07 19:53:01,0,failed,0,US,0,31


In [ ]:
df['state'].value_counts()

In [15]:
# select our target 
target = 'state'

# make train without our target or id
train_features = train.drop(columns=[target])

# make numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# make a cardinality feature to help filter
cardinality = train_features.select_dtypes(exclude='number').nunique()

# get a list of relevant categorical data
categorical_features = cardinality[cardinality <=50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features
print(features)

['ID', 'backers', 'length_of_campaign', 'main_category', 'currency', 'country']


In [ ]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
#X_test = test[features]

In [ ]:
 !pip install category_encoders==2.*

In [16]:
df['goal'].dtypes

dtype('O')

# NLP Model

In [2]:
import pandas as pd

df = pd.read_csv('kickstarter - Sheet1.csv')

print(df.shape)
df

(999, 13)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 0:20:50,45000,2013-01-12 0:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 4:24:11,5000,2012-03-17 3:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 1:00:00,19500,2015-07-04 8:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,1005848496,The 1st Motion Tracking DIY Smart Home Securit...,Gadgets,Technology,USD,2016-12-31 3:54:32,25000,2016-11-01 2:54:32,58978,live,397,US,7235
995,1005862247,Veterans,Photography,Photography,USD,2012-08-15 6:00:00,5000,2012-07-09 5:39:06,5637,successful,87,US,5637
996,1005863674,MY VERY FIRST KICKSTARTER,Movie Theaters,Film & Video,USD,2014-08-12 20:08:35,6500,2014-07-13 20:08:35,27,failed,2,US,27
997,1005870420,This Song Is About You,Music,Music,USD,2014-07-11 23:35:00,40000,2014-06-11 23:35:00,0,failed,0,US,0


In [5]:
df['words'] = df['name'] + df['category']

In [7]:
df['words'] = df['words'] + df['main_category']

In [9]:
sentences = df['words']
y = df['state']

In [18]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(
    sentences, y, test_size=0.25, random_state=42)

In [13]:
df['category'].value_counts()

Product Design    54
Documentary       50
Music             43
Film & Video      39
Video Games       37
                  ..
Calendars          1
Animals            1
Print              1
Footwear           1
Stationery         1
Name: category, Length: 124, dtype: int64

In [17]:
df['main_category'].unique()

array(['Publishing', 'Film & Video', 'Music', 'Food', 'Design', 'Crafts',
       'Games', 'Comics', 'Fashion', 'Theater', 'Art', 'Photography',
       'Technology', 'Dance', 'Journalism'], dtype=object)

In [16]:
df['category'].unique()

array(['Poetry', 'Narrative Film', 'Music', 'Film & Video', 'Restaurants',
       'Food', 'Drinks', 'Product Design', 'Documentary', 'Nonfiction',
       'Indie Rock', 'Crafts', 'Games', 'Design', 'Comic Books',
       'Art Books', 'Fashion', 'Theater', 'Comics', 'DIY', 'Webseries',
       'Food Trucks', 'Public Art', 'Illustration', 'Photography', 'Pop',
       'Tabletop Games', 'Art', 'Family', 'Fiction', 'Accessories',
       'Rock', 'Hardware', 'Weaving', 'Gadgets', 'Web', 'Jazz',
       'Ready-to-wear', 'Festivals', 'Video Games', 'Anthologies',
       'Publishing', 'Shorts', 'Electronic Music', 'Radio & Podcasts',
       'Apps', 'Cookbooks', 'Apparel', 'Metal', 'Hip-Hop', 'Periodicals',
       'Dance', 'Technology', 'Painting', 'Software', 'World Music',
       'Photobooks', 'Drama', 'Architecture', 'Young Adult',
       'Mobile Games', 'Flight', 'Fine Art', 'Action', 'Playing Cards',
       'Makerspaces', 'Punk', "Children's Books", 'People', 'Ceramics',
       'Fabrication Tool

In [20]:
# Import the tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate and fit the tf-idf vectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range = (2,2))
vectorizer.fit(sentences_train)

# Vectorize the training and testing data
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

# Display the properties of the vectorized text
X_train

<749x2760 sparse matrix of type '<class 'numpy.float64'>'
	with 2933 stored elements in Compressed Sparse Row format>

In [23]:
# Import the classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Instantiate and fit a model
classifier = RandomForestClassifier()

classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.54


In [22]:
from sklearn.pipeline import Pipeline

# Define the Pipeline
pipe = Pipeline([('vect', vectorizer), # vectorizer
                 ('clf', classifier) # classifier
                ])

# Define the parameter space for the grid serach
parameters = {'max_depth': [1, 10, 1000000]} 


# Implement a grid search with cross-validation
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(sentences, y);

# Print out the best score
grid_search.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    4.4s finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5375276381909548